In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

def analyze_true_gradients(image_path):
    # Load image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Convert to grayscale for analysis
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Step 1: Identify edges (sharp transitions between solid colors)
    # Use Canny edge detector with appropriate thresholds
    edges = cv2.Canny(gray, 100, 200)
    
    # Step 2: Create a mask to exclude areas near edges
    # Dilate the edges to create a buffer zone
    kernel = np.ones((5, 5), np.uint8)
    edge_mask = cv2.dilate(edges, kernel, iterations=2)
    
    # Invert the mask to get areas away from edges
    gradient_area_mask = 255 - edge_mask
    
    # Step 3: Calculate gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(sobelx**2 + sobely**2)
    
    # Step 4: Apply the mask to exclude edge areas
    # Only consider gradients in non-edge areas
    masked_gradients = np.copy(gradient_magnitude)
    masked_gradients[edge_mask > 0] = 0
    
    # Step 5: Analyze true gradients (smooth transitions)
    # Calculate statistics only on non-edge areas
    if np.sum(gradient_area_mask > 0) > 0:  # If we have non-edge areas
        true_gradient_mean = np.sum(masked_gradients) / np.sum(gradient_area_mask > 0)
        nonzero_gradients = masked_gradients[gradient_area_mask > 0]
        true_gradient_max = np.max(nonzero_gradients) if nonzero_gradients.size > 0 else 0
    else:
        true_gradient_mean = 0
        true_gradient_max = 0
    
    # Step 6: Calculate the percentage of truly gradient pixels
    # First, threshold the masked gradients to identify significant gradients
    gradient_threshold = 10  # Adjust based on your needs
    significant_gradients = (masked_gradients > gradient_threshold).astype(np.uint8)
    gradient_percentage = 100 * np.sum(significant_gradients) / (img.shape[0] * img.shape[1])
    
    # Step 7: Calculate the edge percentage
    edge_percentage = 100 * np.sum(edges > 0) / (img.shape[0] * img.shape[1])
    
    # Step 8: Check for solid color regions
    # Calculate standard deviation in each segment
    # Use a simple approach to segment the image
    # Normalize the gradient magnitude for visualization
    normalized_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    masked_magnitude = cv2.normalize(masked_gradients, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    # Print results
    print("=== IMAGE ANALYSIS RESULTS ===")
    print(f"Edge Percentage: {edge_percentage:.2f}%")
    print(f"True Gradient Percentage: {gradient_percentage:.2f}%")
    print(f"Average True Gradient (excluding edges): {true_gradient_mean:.2f}")
    print(f"Maximum True Gradient (excluding edges): {true_gradient_max:.2f}")
    
    # Determine if the image is primarily solid colors or has true gradients
    if edge_percentage > 1.0 and gradient_percentage < 5.0:
        print("\nCONCLUSION: Image primarily consists of solid colors with sharp boundaries.")
        print("Low true gradient score indicates minimal smooth transitions between colors.")
    elif gradient_percentage > 10.0:
        print("\nCONCLUSION: Image contains significant gradual transitions (true gradients).")
    else:
        print("\nCONCLUSION: Image contains a mix of solid colors and some gradual transitions.")

    
    return {
        'edge_percentage': edge_percentage,
        'true_gradient_percentage': gradient_percentage,
        'true_gradient_mean': true_gradient_mean,
        'true_gradient_max': true_gradient_max
    }

# Example usage
image_path = 'media\e4dd81ea-02af-4bdc-86b7-6b22b614997f.jpg'
results = analyze_true_gradients(image_path)

=== IMAGE ANALYSIS RESULTS ===
Edge Percentage: 6.34%
True Gradient Percentage: 49.73%
Average True Gradient (excluding edges): 36.47
Maximum True Gradient (excluding edges): 198.01

CONCLUSION: Image contains significant gradual transitions (true gradients).


In [ ]:
=== IMAGE ANALYSIS RESULTS ===
Edge Percentage: 8.40%
True Gradient Percentage: 5.56%
Average True Gradient (excluding edges): 4.42
Maximum True Gradient (excluding edges): 199.00

CONCLUSION: Image contains a mix of solid colors and some gradual transitions.






=== IMAGE ANALYSIS RESULTS ===
Edge Percentage: 8.40%
True Gradient Percentage: 5.56%
Average True Gradient (excluding edges): 4.42
Maximum True Gradient (excluding edges): 199.00

CONCLUSION: Image contains a mix of solid colors and some gradual transitions.




=== IMAGE ANALYSIS RESULTS ===
Edge Percentage: 4.83%
True Gradient Percentage: 46.99%
Average True Gradient (excluding edges): 21.17
Maximum True Gradient (excluding edges): 179.12

CONCLUSION: Image contains significant gradual transitions (true gradients).